In [1]:
import pandas as pd
import numpy as np

import pickle
import glob


In [75]:
path = '/home/sdeng/data/icews/events.1991.201703.country/icews_events_THA.json'
df = pd.read_json(path,lines=True)
print('df',len(df))
def getRoot(x):
    x = int(x)
    if len(str(x)) == 4: # 1128
        return x // 100
    elif len(str(x)) == 3:
        if x // 10 < 20: # 190
            return x // 10
        else:
            return x // 100
    else:
        return x // 10

df = df.loc[df['CAMEO Code'] != '---'] 
df['RootEventCode'] = df['CAMEO Code'].apply(lambda x: getRoot(x) )
 
print(df.columns)

Index(['Province', 'Publisher', 'Source Country', 'District', 'City',
       'Country', 'CAMEO Code', 'Target Name', 'Source Sectors', 'Event Date',
       'Source Name', 'Target Sectors', 'Intensity', 'Story ID', 'Latitude',
       'Event Text', 'Longitude', 'Target Country', 'Sentence Number',
       'Event ID', 'RootEventCode'],
      dtype='object')


In [92]:

df_acled_path = '/home/sdeng/data/ACLED/1900-01-01-2021-09-22-Thailand.csv'
df_acled = pd.read_csv(df_acled_path,sep=';')
df_acled['event_date'] = pd.to_datetime(df_acled['event_date'])
print('df_acled',len(df_acled))
print(df_acled.columns)
df_acled.head()

df_acled_date = df_acled.loc[(df_acled['event_date']>='2010-01-01')& (df_acled['event_date']<='2017-03-26')]
print('df_acled_date',len(df_acled_date))

df_acled_date_violence = df_acled_date.loc[(df_acled['event_type'].isin(['Protests','Riots','Violence against civilians']))]
print('df_acled_date_violence',len(df_acled_date_violence))
df_acled_date_violence['event_type'].value_counts()


df_acled 9574
Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')
df_acled_date 6280
df_acled_date_violence 2062


Protests                      1096
Violence against civilians     849
Riots                          117
Name: event_type, dtype: int64

In [102]:
df['Source Country'].value_counts()
df
# df['Publisher'].value_counts()
# df['Source Sectors'].values[1]
# df[['Intensity','CAMEO Code']].value_counts()[:60]
# df[['Intensity']].value_counts()
df2010 = df.loc[(df['Event Date']>='2010-01-01') & (df['Event Date']<='2017-03-26')]
print('df2010',len(df2010))


# df2010.head()
df2010.loc[(df2010['RootEventCode']==14)&(df2010['Sentence Number']<=3)]

df2010 77479


,Province,Publisher,Source Country,District,City,Country,CAMEO Code,Target Name,Source Sectors,Event Date,...,Target Sectors,Intensity,Story ID,Latitude,Event Text,Longitude,Target Country,Sentence Number,Event ID,RootEventCode
139969,Changwat Nakhon Ratchasima,Thai News Service,Thailand,,Nakhon Ratchasima,Thailand,141,Unspecified Actor,"Social,General Population / Civilian / Social",2010-01-11,...,Unspecified,-6.5,7149884,14.9707,Demonstrate or rally,102.102,,2,15038070,14
139983,Changwat Nakhon Ratchasima,Thai News Service,Thailand,,Nakhon Ratchasima,Thailand,141,Government (Thailand),Elite,2010-01-11,...,Government,-6.5,7149932,14.9707,Demonstrate or rally,102.102,Thailand,2,15038088,14
139997,Changwat Nakhon Ratchasima,Bangkok Post,Thailand,,Nakhon Ratchasima,Thailand,145,Unspecified Actor,"Protestors / Popular Opposition / Mobs,Dissident",2010-01-11,...,Unspecified,-7.5,7150987,14.9707,"Protest violently, riot",102.102,,1,15038222,14
139998,Changwat Nakhon Ratchasima,Bangkok Post,Thailand,,Nakhon Ratchasima,Thailand,141,Cabinet / Council of Ministers / Advisors (Tha...,"Protestors / Popular Opposition / Mobs,Dissident",2010-01-11,...,"Government,Cabinet,Executive",-6.5,7150987,14.9707,Demonstrate or rally,102.102,Thailand,2,15038221,14
140000,Changwat Nakhon Ratchasima,Bangkok Post,Thailand,,Nakhon Ratchasima,Thailand,141,Unspecified Actor,"Protestors / Popular Opposition / Mobs,Dissident",2010-01-11,...,Unspecified,-6.5,7151149,14.9707,Demonstrate or rally,102.102,,3,15038251,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216813,,Bangkok Post,Thailand,,,Thailand,141,Unspecified Actor,"Social,General Population / Civilian / Social",2017-03-07,...,Unspecified,-6.5,43655947,13.7500,Demonstrate or rally,100.517,,2,27285286,14
216823,,Bangkok Post,Thailand,,,Thailand,141,Unspecified Actor,"Dissident,Protestors / Popular Opposition / Mobs",2017-03-07,...,Unspecified,-6.5,43665372,13.7500,Demonstrate or rally,100.517,,3,27289552,14
216861,Krung Thep Mahanakhon,Bangkok Post,Thailand,,Bangkok,Thailand,141,Other Authorities / Officials (Thailand),"Social,General Population / Civilian / Social",2017-03-11,...,Government,-6.5,43702474,13.7540,Demonstrate or rally,100.501,Thailand,1,27352103,14
216918,Krung Thep Mahanakhon,Unknown,Thailand,,Bangkok,Thailand,141,Unspecified Actor,"Dissident,Protestors / Popular Opposition / Mobs",2017-03-16,...,Unspecified,-6.5,43741348,13.7540,Demonstrate or rally,100.501,,1,27368378,14


In [93]:
# df2010['event_type'].value_counts()

In [83]:
df2010['City'].value_counts()
# df2010['CAMEO Code'].value_counts()
df2010.loc[df2010['RootEventCode']==14]['City'].value_counts()[:60]

Bangkok                     4349
                            1238
Chiang Mai                    78
Pathum Thani                  62
Victory Monument              48
Songkhla                      41
Nakhon Si Thammarat           33
Nakhon Ratchasima             33
Udon Thani                    29
Khon Kaen                     28
Mueang Nonthaburi             25
Surat Thani                   24
Ban Talat Rangsit             24
Suvarnabhumi                  22
Phuket                        22
Phrao                         21
Phitsanulok                   18
Rayong                        18
Dusit                         18
Din Daeng                     18
Krabi                         16
Chumphon                      16
Samut Prakan                  14
Trang                         13
Ban Ton Krang                 13
Phra Nakhon Si Ayutthaya      12
Ubon Ratchathani              12
Kalasin                       12
Ratchaburi                    12
Phetchaburi                   11
Kanchanabu

In [113]:
tmp_filter = df2010.loc[(df2010['City']=='Bangkok') & (df2010['RootEventCode']==14) & (df2010['Sentence Number']<=1) ]
tmp_filter['Event Date'].value_counts()[:60]

2010-05-14    36
2010-05-15    32
2010-04-12    27
2010-04-10    26
2013-11-25    23
2010-03-14    22
2010-04-07    22
2010-04-28    21
2010-05-19    20
2014-02-18    19
2010-05-17    19
2014-01-13    18
2013-12-26    17
2014-02-14    17
2010-04-09    16
2010-03-12    15
2010-05-16    15
2010-03-16    15
2014-02-19    14
2010-04-08    14
2010-04-13    14
2013-11-26    13
2013-12-09    12
2010-05-18    12
2013-11-27    12
2010-03-15    12
2013-12-02    12
2012-11-24    11
2013-11-30    11
2014-03-29    10
2014-01-14    10
2010-03-27    10
2010-03-13    10
2010-04-29    10
2013-08-04     9
2010-04-14     9
2013-12-23     9
2014-01-26     9
2010-04-04     8
2014-02-17     8
2014-01-18     8
2014-01-15     8
2014-01-20     8
2013-11-04     7
2010-04-26     7
2013-12-01     7
2010-04-03     7
2010-05-12     6
2013-11-24     6
2010-04-15     6
2010-04-06     6
2010-04-11     6
2010-05-13     6
2013-12-19     6
2010-03-26     6
2010-03-20     6
2010-04-05     6
2010-05-03     6
2010-04-16    

In [67]:
df_acled[['location','admin1','admin2','admin3']]
df_acled['admin1'].value_counts()
# df_acled['admin2'].value_counts()

Pattani            2252
Narathiwat         2177
Bangkok            1542
Yala               1443
Songkhla            255
                   ... 
Samut Songkhram       5
Sing Buri             4
Suphan Buri           4
Amnat Charoen         3
Bueng Kan             3
Name: admin1, Length: 76, dtype: int64

In [98]:
df_acled_date_violence_city = df_acled_date_violence.loc[ (df_acled['admin1']=='Bangkok')]
print('df_acled_date_violence_city',len(df_acled_date_violence_city))
df_acled_date_violence_city['event_type'].value_counts()


df_acled_date_violence_city 605


Protests                      540
Riots                          51
Violence against civilians     14
Name: event_type, dtype: int64

In [127]:
# df_acled_date_violence_city['event_date'].value_counts()[:60]

# get word frequency by day
from gensim.models.ldamulticore import LdaMulticore,LdaModel
from gensim.test.utils import common_texts
# from gensim.corpora.dictionary import Dictionary
# from gensim.test.utils import common_corpus, common_dictionary
# lda = LdaMulticore(common_corpus, id2word=common_dictionary, num_topics=10)


In [147]:
# common_dictionary
# common_corpus
# common_dictionary = Dictionary(common_texts)
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
common_texts
lda = LdaModel(common_corpus, num_topics=10)

In [152]:
common_corpus
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [158]:

# lda.print_topics(20, num_words=20)
mytext =  [['I','am','working','on','a','survey', 'that', 'is', 'focus', 'on', 'event','prediction'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees','are','beautiful'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]
my_dictionary = Dictionary(mytext)

my_corpus = [my_dictionary.doc2bow(text) for text in mytext] # doc2idx
my_corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 2),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1)],
 [(11, 1), (12, 1), (13, 1), (14, 1)],
 [(11, 1), (13, 2), (15, 1)],
 [(14, 1), (16, 1), (17, 1)],
 [(18, 1), (19, 1), (20, 1)],
 [(20, 1), (21, 1)],
 [(20, 1), (21, 1), (22, 1)],
 [(8, 1), (21, 1), (22, 1)]]

In [141]:
lda.get_topics().shape
lda.get_topic_terms(0)
lda.get_term_topics(1)
# lda.get_document_topics(1)
lda.show_topic(1)

#todo ge topics? for a country, check words

[('5', 0.18749832),
 ('7', 0.18749519),
 ('0', 0.09821461),
 ('6', 0.09821399),
 ('4', 0.09821369),
 ('3', 0.09821338),
 ('2', 0.09821271),
 ('8', 0.09821131),
 ('9', 0.008933482),
 ('10', 0.008931831)]